In [1]:
import numpy as np
import pandas as pd

In [2]:
url = "https://drive.google.com/drive/u/0/folders/1akT7jbXYy2-zLHEtnK3q6vqcQ-JdRO9H"

In [3]:
url = "https://drive.google.com/file/d/1u98UkVheVIcyXx-ir1Gp5fggsu2V3ztC/view?usp=sharing"

In [4]:
#url='https://drive.google.com/file/d/0B6GhBwm5vaB2ekdlZW5WZnppb28/view?usp=sharing'
#url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)

In [5]:
df

,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""BfoehudmNFdKVl54+SaBKw"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>alerts.csv - Google Drive</title><meta property=""og:title"" content=""alerts.csv""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1u98UkVheVIcyXx-ir1Gp5fggsu2V3ztC/view?usp=sharing&amp;usp=embed_facebook""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans_old:300",400,500,"700"" nonce=""BfoehudmNFdKVl54+SaBKw""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.JW9iDIQ97HI.L.X.O/am=IA/d=0/rs=AO0039tT-W-uRbQhHLOKda6ysCGEjbnmZw"" nonce=""BfoehudmNFdKVl54+SaBKw""><script nonce=""BZMbQ+OMOKMSDnCi2rPjaA"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nfd""",docs-shdn:0,"docs-tfh:""""",info_params:{},docos-eddmh:false,...,null.188,null.189,-Ui0YZ-mKf-rwbkPlYC74Ao.1,0.74,0].9,[[null.2,null.190,null.191,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.ObZik2v2gsg.O/rt=j/m=qabr,q_dnp,qapid/exm=qaaw,qadd,qaid,qein,qhaw,qhbr,qhch,qhga,qhid,qhin,qhpr/d=1/ed=1/rs=AA2YrTs2PfrCrMqqnFEvLQxiYSH-jsPO1w]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
0,try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,var ea,ia,ta,Aa,Fa,Ha,Ia,Ja,Ka,La,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"_.fa=function(){return _.p(""Safari"")&&!(ea()||...",b){return 0<=(0,_.ka)(a,b)};,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,_.ma=function(a){var b=a.length;if(0<b){for(va...,d=0;d<b;d++)c[d]=a[d];return c}return[]};_.na=...,{ze:{value:1,configurable:!0,writable:!0,enumerable:!1}}));return a};ta=function(a){ret...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"_.xa=function(a){switch(typeof a){case ""number...",b){if(null!=a)return Array.isArray(a)||ta(a)?_...,b):b(a)};,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,(function(){var a=_.Ld();if(_.F(a,18))Nj();else{var b=_.E(a,"19)||0;window.addEventListener(""load""",function(){window.setTimeout(Nj,b)})}})();,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
172,}catch(e){_._DumpException(e)},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
173,})(this.gbar_);,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
174,// Google Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
